In [1]:
import numpy as np
import os
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

import torchtime
from torchtime.models import InceptionTime
from torchtime.datasets import TimeSeriesDataset
from torchtime.transforms import LabelEncoder, ToTensor, Compose, Pad
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
from torchmetrics import Accuracy

In [2]:
panda = False

if panda: 
    filepath = './Data/CombinedData/combined_data_training_panda.pickle'
    model_path = "./Models/panda_ft_classifier"
    test_dataset_path = './Data/CombinedData/combined_data_testing_panda.pickle'
    PAD = Pad(300)
else:
    filepath = './Data/CombinedData/combined_data_training_ilosa.pickle'
    model_path = "./Models/ilosa_ft_classifier"
    test_dataset_path = './Data/CombinedData/combined_data_testing_ilosa.pickle'
    PAD = Pad(900)
    

In [3]:
def collate_fn(batch):
    print(len(batch))
    return tuple(zip(*batch))

class ForceDataset(TimeSeriesDataset):
    
    def __init__(self, root: str, transforms = None, transform = None):
        self.root = root
        super(ForceDataset, self).__init__(self.root, transforms, transform, Compose([LabelEncoder(['left', 'right'])]))

        self.__load()
    
    def dim(self):
        return self.data.shape[0]
    
    def __load(self):
        data = np.array(np.load(self.root, allow_pickle=True), dtype=object)
        self.data = data[:, 1]
        self.labels = data[:, 0]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        x = np.array(self.data[index], dtype=np.float32).T
        return self.transforms(x, self.labels[index])

In [4]:
dataset = ForceDataset(filepath,
                       transform=Compose([ToTensor()]))

dataset.dim

<bound method ForceDataset.dim of Dataset ForceDataset
    Number of datapoints: 63
    Root location: ./Data/CombinedData/combined_data_training_ilosa.pickle
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Target transform: Compose(
                      LabelEncoder(
                  	Encoding Table: {
                  		left: 0
                  		right: 1
                  	}
                  )
                  )>

In [5]:
dataloader = DataLoader(dataset, batch_size=3, shuffle=True, pin_memory=True, num_workers=1)

In [6]:
class ForceClassifier(pl.LightningModule):
    
    def __init__(self, root, batch_size=3, learning_rate=1e-3):
        super().__init__()
        self.root = root
        self.generator=torch.Generator().manual_seed(42)
        self.ce_loss = torch.nn.CrossEntropyLoss()
        self.learning_rate = learning_rate
        self.batch_size = batch_size

        self.train_acc = Accuracy()
        self.valid_acc = Accuracy()
        self.test_acc = Accuracy()
    
    def setup(self, stage):
        dataset = ForceDataset(self.root, transform=Compose([ToTensor(), PAD]))
        self.train_dataset, self.test_dataset = torch.utils.data.random_split(dataset, [50, 13])
        self.model = InceptionTime(n_inputs=6, n_classes=2)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, pin_memory=True,
                          num_workers=1)
    
    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True,
                   num_workers=1)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True,
                   num_workers=1)
    
    def training_step(self, train_batch, batch_index):
        x, y = train_batch
        x_hat = self.model(x)
        loss = self.ce_loss(x_hat, y)
        self.log('train_loss', loss)
        self.train_acc(x_hat, y)
        self.log("train_acc", self.train_acc)
        return loss
    
    def validation_step(self, val_batch, val_index):
        x, y = val_batch
        x_hat = self.model(x)
        loss = self.ce_loss(x_hat, y)
        self.log('val_loss', loss)
        self.valid_acc(x_hat, y)
        self.log("val_acc", self.valid_acc)

    def test_step(self, test_batch, test_index):
        x, y = test_batch
        x_hat = self.model(x)
        loss = self.ce_loss(x_hat, y)
        self.log('val_loss', loss)
        self.test_acc(x_hat, y)
        self.log("test_acc", self.test_acc)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


In [7]:
classifier = ForceClassifier(filepath)

In [8]:
trainer = pl.Trainer(max_epochs=100)

/home/damnghost/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/damnghost/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
trainer.fit(classifier)


  | Name      | Type             | Params
-----------------------------------------------
0 | ce_loss   | CrossEntropyLoss | 0     
1 | train_acc | Accuracy         | 0     
2 | valid_acc | Accuracy         | 0     
3 | test_acc  | Accuracy         | 0     
4 | model     | InceptionTime    | 387 K 
-----------------------------------------------
387 K     Trainable params
0         Non-trainable params
387 K     Total params
1.550     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/damnghost/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/damnghost/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/damnghost/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers wh

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [10]:
trainer.test()

/home/damnghost/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1386: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at /home/damnghost/Documents/STUDY/HBRS/RnD/RnD_Project/Experimentation/FT_DATA/test/lightning_logs/version_19/checkpoints/epoch=99-step=1700.ckpt
Loaded model weights from checkpoint at /home/damnghost/Documents/STUDY/HBRS/RnD/RnD_Project/Experimentation/FT_DATA/test/lightning_logs/version_19/checkpoints/epoch=99-step=1700.ckpt
/home/damnghost/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        val_loss           0.004658487159758806
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss': 0.004658487159758806, 'test_acc': 1.0}]

# Inference from saved model 

In [11]:
save_model = torch.save(classifier.model, model_path)

In [12]:
model = torch.load(model_path)
model.eval()

InceptionTime(
  (blocks): ModuleList(
    (0): Sequential(
      (Inception_0): Inception(
        (bottleneck): Conv1d(6, 32, kernel_size=(1,), stride=(1,), padding=same, bias=False)
        (conv_layers): ModuleDict(
          (Conv1D_0): BasicConv1d(
            (conv): Conv1d(32, 32, kernel_size=(31,), stride=(1,), padding=same, bias=False)
            (bn): BatchNorm1d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (activation): Identity()
          )
          (Conv1D_1): BasicConv1d(
            (conv): Conv1d(32, 32, kernel_size=(15,), stride=(1,), padding=same, bias=False)
            (bn): BatchNorm1d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (activation): Identity()
          )
          (Conv1D_2): BasicConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=same, bias=False)
            (bn): BatchNorm1d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
     

In [13]:
# Function to test the model 
def test(model, test_loader): 
     
    running_accuracy = 0 
    total = 0 
 
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            print(inputs.shape)
            print(outputs.shape)
            outputs = outputs.to(torch.float32) 
            print(f"Outputs: {outputs}")
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
            print(f"Predicted Max: {predicted}")
            print(f"Predicted: {predicted_outputs}")
            total += outputs.size(0) 
            running_accuracy += (predicted == outputs).sum().item() 
 
        print('Accuracy of the model based on the test set of inputs is: %d %%' % (100 * running_accuracy / total))    
 


In [14]:
dataset = ForceDataset(test_dataset_path,
                       transform=Compose([ToTensor()]))

dataloader = DataLoader(dataset, batch_size=1, shuffle=True, pin_memory=True, num_workers=1)

test(model, dataloader)

torch.Size([1, 6, 898])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicted: tensor([[-4.6177,  4.5735]])
torch.Size([1, 6, 764])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicted: tensor([[-5.1782,  5.0076]])
torch.Size([1, 6, 904])
torch.Size([1])
Outputs: tensor([0.])
Predicted Max: tensor([0])
Predicted: tensor([[ 2.5420, -2.5300]])
torch.Size([1, 6, 813])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicted: tensor([[-5.0641,  5.0511]])
torch.Size([1, 6, 794])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicted: tensor([[-5.1556,  5.0613]])
torch.Size([1, 6, 794])
torch.Size([1])
Outputs: tensor([0.])
Predicted Max: tensor([0])
Predicted: tensor([[ 4.7455, -4.7628]])
torch.Size([1, 6, 805])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicted: tensor([[-5.2883,  5.2199]])
torch.Size([1, 6, 906])
torch.Size([1])
Outputs: tensor([1.])
Predicted Max: tensor([1])
Predicte